In [50]:

import openai
import langchain
import pinecone
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI


In [51]:
from dotenv import load_dotenv

load_dotenv()



True

In [52]:
import os

In [53]:
# Read the document 
def read_document(file_path):
    file_loader = PyPDFDirectoryLoader(file_path)
    file_loaders = file_loader.load()
    return file_loaders

In [54]:
doc=read_document('documents/')
len(doc)

9

In [64]:
doc

[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-04-03T16:00:17+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-04-03T16:00:17+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'documents\\The_IT_Job_market_situation_in_2025_report.pdf', 'total_pages': 9, 'page': 0, 'page_label': '1'}, page_content='Research Report\n The IT Job market situation in 2025\n Generated by DeepResearch AI\n April 03, 2025'),
 Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-04-03T16:00:17+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-04-03T16:00:17+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'documents\\The_IT_Job_market_situation_in_2025_report.pdf', 'total_pages': 9, 'page': 1, 'page_label': '2'}, page_content=''),
 Document(metad

In [55]:
# Divide the document into smaller chunks

def chunk_document(doc):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )
    docs = text_splitter.split_documents(doc)
    return docs

In [56]:
documents=chunk_document(doc)
len(documents)

12

In [57]:
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec

# 1. create the client
pc = Pinecone(
    api_key="pcsk_2FkBj9_FUWpNWG3VEA5M9udA8goVuBV8hWmUdaipdZShFWBjtVAywdshJsDs3125Yr1VXB"
)
index_name = "langchain-vector"

In [58]:
texts = ["Cats are great pets.", "Pine trees stay green all year."]
ids   = ["doc1", "doc2"]


In [ ]:
# 1. embed
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(texts).tolist()      # list of 768-dim float lists
# 2. create index
# delete if it exists
if index_name in pc.list_indexes().names():
    pc.delete_index(name=index_name)

# recreate with 384 dimensions
pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)
index = pc.Index(index_name)


# 3. loop, embed, upsert
vectors = []
for doc in documents:
    text = doc.page_content.strip()
    if not text:
        continue
    vec = model.encode(text).tolist()
    # e.g. "report_page_4" or use the source+page
    vid = f"{os.path.basename(doc.metadata['source'])}_p{doc.metadata['page']}"
    vectors.append((vid, vec))

# batch upsert
index.upsert(vectors=vectors)

# 4. query
query_vec = model.encode("Evergreen trees").tolist()
results = index.query(vector=query_vec, top_k=2)
print(results)



{'matches': [], 'namespace': '', 'usage': {'read_units': 1}}


In [60]:
from langchain.chains.question_answering import load_qa_chain       
from langchain import OpenAI

In [61]:
llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0)
chain = load_qa_chain (llm, chain_type="stuff")

d:\Projects\E2E_LLM_APP\venv\Lib\site-packages\langchain_community\llms\openai.py:255: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
d:\Projects\E2E_LLM_APP\venv\Lib\site-packages\langchain_community\llms\openai.py:1089: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
C:\Users\user\AppData\Local\Temp\ipykernel_9184\3706586090.py:2: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chai

In [62]:
# search answer from the vector database

def retrieve_answer(query):
    # 1. create the client
    pc = Pinecone(
        api_key="pcsk_2FkBj9_FUWpNWG3VEA5M9udA8goVuBV8hWmUdaipdZShFWBjtVAywdshJsDs3125Yr1VXB"
    )
    index_name = "langchain-vector"

    # 2. embed
    model = SentenceTransformer("all-MiniLM-L6-v2")
    query_vec = model.encode(query).tolist()      # list of 768-dim float lists

    # 3. query
    index = pc.Index(index_name)
    results = index.query(vector=query_vec, top_k=2)
    
    return results  

In [63]:
my_querry= "Skills and Qualifications in Demand"

answer =retrieve_answer(my_querry)
print(answer)

{'matches': [], 'namespace': '', 'usage': {'read_units': 1}}
